# This is an analysis of what could be Mint Mobile's pricing scheme
It starts from an observation that for the 10GB plan, which is the plan I'm currently on, the price of the months increase (the monthly gap price) is lower than the plan's current monthly price.
If the monthly gap price is the actual business cost of that plan, then mint is making a profit.
It made me wonder how much of a profit there is and what is the logic of the pricing scheme, starting with the 3-month plan for each data package.

In [1]:
import numpy as np

## We start with our data from mint app:

In [95]:
# our data:         4gb 10gb 25gb unltd
prices = np.array([[75, 105, 135, 120], # 3 monnths
                   [120, 150, 210, 210],# 6 months 
                   [180, 240, 300, 360]]) # 12 months
print("Current total prices (month x package):\n",prices)
prices_per_month = prices/np.array([[3],[6],[12]])
print("Current monthly prices (month x package):\n",prices_per_month)

Current total prices (month x package):
 [[ 75 105 135 120]
 [120 150 210 210]
 [180 240 300 360]]
Current monthly prices (month x package):
 [[25. 35. 45. 40.]
 [20. 25. 35. 35.]
 [15. 20. 25. 30.]]


## We then calculate the monthly gap prices
We subtract the 6-month and 12-month plans' total prices from that of 3-month for each data package
The monthly gap price is the the difference divided by how many months in the gap between the plans. The gap between 6- and 3-month planse are 3 months, between 12- and 3- are 9 months.

In [106]:
actual_monthly = (prices[1:3]-prices[0])/np.array([[3],[9]])
print("Monthly gap price:\n", actual_monthly)

Monthly gap price:
 [[15.         15.         25.         30.        ]
 [11.66666667 15.         18.33333333 26.66666667]]


## If monthly gap price is the actual price for each month of the entire plan...
... then the profit Mint is earning per plan per data package would be the difference between the prices under current monthly pricing and what would have been under the monthly gap prices.

Since we use the gap price from initial 3-month plan, the profits are calculated for the 6-month and 12-month plans

In [111]:
current_profit = prices[1:3] - actual_monthly*np.reshape([6,12],(2,1)) # assuming that this is the actual price
                                                                        # for all months of a plan
print("Current profit for 6- and 12-month plans for all 4 data packages:")
print("6-month plan profit  ($):", current_profit[0])
print("12-month plan profit ($):", current_profit[1])

Current profit for 6- and 12-month plans for all 4 data packages:
6-month plan profit  ($): [30. 60. 60. 30.]
12-month plan profit ($): [40. 60. 80. 40.]


## We can then derive how the initial 3-month plan price is set
Using the profits and assumed original monthly prices calculated above, we reverse engineer the total prices and the price of the 3-month plan.

Mint must set a total price that takes into account the actual cost and the desired profit. 
The three month bears all the price differences, so the desired profit must be divided by three to get the target monthly gap.

Thus the target monthly price for the 3-month plan would be the actual monthly price plus the target monthly gap.

We use the 10GB plan as an example.

In [109]:
plans = ["4gb", "10gb", "25gb", "unlimited"]
desired_earning = np.array([60, 60])
plan_actual_monthly  = actual_monthly[:,plans.index("10gb")] # this actual monthly is based on current pricing but
                                                             # I'm guessing it's set so that you can have the same
                                                             # starting 3-month price for both 6 and 12-month plans

price_to_set = plan_actual_monthly * [6,12] + desired_earning
print("Price to set to get desired profit:",price_to_set)
target_three_month_monthly_gap = desired_earning/3
print("What the target monthly gap must be:",target_three_month_monthly_gap)
target_three_month_price = target_three_month_monthly_gap+plan_actual_monthly
print("Target 3-month plan price:",target_three_month_price)

Price to set to get desired profit: [150. 240.]
What the target monthly gap must be: [20. 20.]
Target 3-month plan price: [35. 35.]


## We have recorved the original 3-month plan price
Since both the 6- and 12-month share the same 3-month starting price, the profit and monthly price must be set so that the calculation lead to the same number, as shown above. This would explain the differences in the calculated actual monthly price and profits outside the 10GB plan. 

If we put on the hat of Mint CFO, we want a 3-month plan monthly price that is competitive against all current carriers and also maximizes the profit with reasonable expectation for cost of both the 6- and 12- month plans.

## So this is after all an optimization problem. 

Let y be the target 3-month monthly price. x1 be monthly cost for 6-month plan, x2 for 12-month plan.
The optimization problem is then

$$max_{x_1, x_2, y} [(3y+3x_1)-6x_1] + [(3y+9x_2)-12x_2]$$
s.t. $$y \leq 35$$
$$x_1 > 10$$
$$x_2 > 10$$

We want the two main terms to be relatively close together. The bounds for y and x1, x2 are educated guesses.